In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/data-customers/customers.csv


In [2]:
pip install pyspark

     |████████████████████████████████| 204.2 MB 35 kB/s 
     |████████████████████████████████| 198 kB 40.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612244 sha256=7654ea1e846c026f7ed1fe29d534dab14a8355551d6193d578a7846f6cfd1816
  Stored in directory: /root/.cache/pip/wheels/5e/34/fa/b37b5cef503fc5148b478b2495043ba61b079120b7ff379f9b
Successfully built pyspark
You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
#Mari kita import Packagenya.
from pyspark.sql import SparkSession
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler

In [4]:
#Buat Sesi terlebih dahulu

appName="Cluatering dengan Spark"
spark=SparkSession.builder.master("local").appName(appName).getOrCreate()
sc=spark.sparkContext
sc


<SparkContext master=local appName=Cluatering dengan Spark>

In [5]:
#Load dataset
customers=spark.read.csv('../input/data-customers', inferSchema=True, header=True)
customers.show(5)

+---------------+---+-------------+-----------+------+-------------+--------------+---------+----------+---------+----+
|   CustomerName|Age|MaritalStatus|IncomeRange|Gender|TotalChildren|ChildrenAtHome|Education|Occupation|HomeOwner|Cars|
+---------------+---+-------------+-----------+------+-------------+--------------+---------+----------+---------+----+
|    Aaron Adams| 42|            0|      50000|     0|            0|             0|        3|         2|        1|   1|
|Aaron Alexander| 40|            1|      50000|     0|            0|             0|        2|         2|        1|   2|
|    Aaron Allen| 63|            0|      25000|     0|            2|             1|        2|         1|        1|   2|
|    Aaron Baker| 56|            1|      50000|     0|            4|             2|        2|         2|        1|   2|
|   Aaron Bryant| 72|            0|      75000|     0|            4|             0|        4|         4|        1|   2|
+---------------+---+-------------+-----

In [6]:
#Melakukan feature engineering
assembler=VectorAssembler(inputCols=["Age","MaritalStatus", "IncomeRange","Gender","TotalChildren","ChildrenAtHome","Education","Occupation","HomeOwner","Cars"], outputCol="features")
train=assembler.transform(customers).select("CustomerName","features")
train.show(truncate=False,n=3)


+---------------+----------------------------------------------+
|CustomerName   |features                                      |
+---------------+----------------------------------------------+
|Aaron Adams    |[42.0,0.0,50000.0,0.0,0.0,0.0,3.0,2.0,1.0,1.0]|
|Aaron Alexander|[40.0,1.0,50000.0,0.0,0.0,0.0,2.0,2.0,1.0,2.0]|
|Aaron Allen    |[63.0,0.0,25000.0,0.0,2.0,1.0,2.0,1.0,1.0,2.0]|
+---------------+----------------------------------------------+
only showing top 3 rows



In [7]:
#Membuat Modelnya disini menggunakan K=5
kmeans=KMeans(featuresCol=assembler.getOutputCol(),predictionCol="cluster", k=5, seed=0)

#Training Model
model=kmeans.fit(train)
print("Model Trained")

Model Trained


In [8]:
#Mencari Nilai Tengah
centers = model.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)

Cluster Centers: 
[5.31013005e+01 4.17180014e-01 2.50000000e+04 4.80492813e-01
 1.41512663e+00 6.08487337e-01 2.31622177e+00 1.45448323e+00
 5.93086927e-01 1.11464750e+00]
[5.53417813e+01 5.72411296e-01 1.00000000e+05 4.97103548e-01
 2.54380883e+00 1.54272266e+00 3.46198407e+00 4.19116582e+00
 7.16509776e-01 1.94532947e+00]
[5.19737441e+01 5.26868545e-01 5.00000000e+04 4.93961141e-01
 1.34552774e+00 4.98337126e-01 3.23035183e+00 2.77927534e+00
 6.62699107e-01 1.14615789e+00]
[5.60711289e+01 5.83804487e-01 7.50000000e+04 5.03921211e-01
 2.17308043e+00 8.16706183e-01 3.73244574e+00 3.92759438e+00
 7.23326646e-01 1.38063104e+00]
[5.82794840e+01 6.22850123e-01 1.50000000e+05 4.79729730e-01
 2.07248157e+00 3.20638821e+00 3.41461916e+00 4.34705160e+00
 6.48648649e-01 3.10995086e+00]


In [9]:
#Prediksi
prediction = model.transform(train)#melakukan prediksi klaster
prediction.groupBy("cluster").count().orderBy("cluster").show()
prediction.select('CustomerName', 'cluster').show(5)#menampilkan 5 data hasil prediksi

+-------+-----+
|cluster|count|
+-------+-----+
|      0| 2922|
|      1| 2762|
|      2| 5713|
|      3| 5483|
|      4| 1628|
+-------+-----+

+---------------+-------+
|   CustomerName|cluster|
+---------------+-------+
|    Aaron Adams|      2|
|Aaron Alexander|      2|
|    Aaron Allen|      0|
|    Aaron Baker|      2|
|   Aaron Bryant|      3|
+---------------+-------+
only showing top 5 rows

